In [1]:
from utils.basicFeature import basicFeature
basicFeature.integrate_all_features("./time-series-data/AABA_2006-01-01_to_2018-01-01.csv")

{'AvgReturn': np.float64(0.00046776334751751563),
 'Volatility': 0.024343510513038555,
 'SharpeRatio': np.float64(0.019215114733225444),
 'MDD': -0.793873790879779,
 'Skewness': np.float64(2.0014214154758183),
 'Kurtosis': np.float64(56.23659815632061)}